Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Libraries imported.


Create Dataframe for Toronto Data - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
def strip_formating(tblRow):
    tblRow = tblRow.text.strip()
    strRow = str(tblRow)
    arrRow = strRow.split('\n')
    return arrRow

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
get_text = requests.get(url).text
soup = BeautifulSoup(get_text,'xml')

NH_tbl = soup.find('table',{'class':'wikitable sortable'})
NH_rw = NH_tbl.find_all('tr')

arr_data = []
for iRow in NH_tbl.find_all('tr'):
    arrRow = strip_formating(iRow)
    arr_data.append(arrRow) 
df = pd.DataFrame(arr_data, columns=['PostalCode', 'Borough', 'Neighborhood'])

Remove rows with Neighborhood = Not assigned, Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 

In [3]:
df1 = df[df.Borough!='Not assigned']
df1 = df1[df1.Borough!='Borough']
df1.reset_index(inplace=True)

If More than one neighborhood can exist in one postal code area combined into one row with the neighborhoods separated with a comma 
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [4]:
tempPC = 'MMM'
tempNH = 'NNN'
for x in range(df1.shape[0]):
    if df1.Neighborhood[df1.index==x].values[0] == 'Not assigned':
        df1.at[x,'Neighborhood'] = df1.Borough[df1.index==x].values[0]
        
    if df1.PostalCode[df1.index==x].values[0] == tempPC:  
        df1.at[x,'Neighborhood'] = tempNH + ', ' + df1.Neighborhood[df1.index==x].values[0]
        df1.at[x-1,'Neighborhood'] = 'Delete'
    tempPC =  df1.PostalCode[df1.index==x].values[0]
    tempNH =  df1.Neighborhood[df1.index==x].values[0]
df2 = df1[df1.Neighborhood!='Delete']    

Reindex


In [5]:
df2.reset_index(inplace=True)
del df2['index']
del df2['level_0']
df2.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Use the .shape method to print the number of rows of your dataframe.

In [6]:
df2.shape

(103, 3)